In [203]:

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier

# from unidecode import unicode
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [204]:
df = pd.read_parquet("clean_datasets/machine_learning.parquet")

In [205]:
df

,titre_str,titre_genres,rating_avg,rating_votes,stars,directors,overview
0,Kate & Leopold,"[Comedy, Fantasy, Romance]",6.4,87897,"[Meg Ryan, Hugh Jackman, Liev Schreiber, Breck...",[James Mangold],When her scientist ex-boyfriend discovers a po...
1,The Fugitive Kind,"[Drama, Romance]",7.1,6914,"[Anna Magnani, Marlon Brando, Joanne Woodward,...",[Sidney Lumet],"Val Xavier, a drifter of obscure origins, arri..."
2,The League of Gentlemen,"[Comedy, Crime, Drama]",7.2,5105,"[Jack Hawkins, Roger Livesey, Nigel Patrick, R...",[Basil Dearden],Involuntarily-retired Colonel Hyde recruits se...
3,Eyes Without a Face,"[Drama, Horror]",7.6,34278,"[Pierre Brasseur, Alexandre Rignault, Alida Va...",[Georges Franju],Dr. Génessier is riddled with guilt after an a...
4,Late Autumn,"[Comedy, Drama]",7.9,5854,"[Setsuko Hara, Keiji Sada, Mariko Okada, Yôko ...",[Yasujirô Ozu],A woman and her daughter are each forced to co...
...,...,...,...,...,...,...,...
7411,They Cloned Tyrone,"[Comedy, Mystery, Sci-Fi]",6.6,33197,"[Kiefer Sutherland, Jamie Foxx, John Boyega, T...",[Juel Taylor],A series of eerie events thrusts an unlikely t...
7412,Dream Horse,"[Biography, Comedy, Drama]",6.9,6589,"[Alan David, Lynda Baron, Owen Teale, Toni Col...",[Euros Lyn],"The inspiring true story of Dream Alliance, an..."
7413,I Care a Lot,"[Comedy, Crime, Drama]",6.4,141372,"[Dianne Wiest, Rosamund Pike, Peter Dinklage, ...",[J Blakeson],A court-appointed legal guardian defrauds her ...
7414,Kaithi,"[Action, Adventure, Crime]",8.5,39546,"[Narain, Karthi, George Maryan, Arjun Das]",[Lokesh Kanagaraj],"Dilli, a convicted criminal, is out on parole ..."


In [206]:
tt = (
    ("stars", "stars"),
    ("titre_genres", "titre_genres"),
    ("directors", "directors"),
)
for t in tt:
    df[t[0]] = df[t[1]].apply(lambda x: ", ".join(map(str, x)).replace(" ", ""))



In [207]:
df[df["titre_str"].str.contains("Spider Man")]

,titre_str,titre_genres,rating_avg,rating_votes,stars,directors,overview


In [208]:
movie = "Spider-Man"

In [209]:
feat = [
    'titre_str',
    'titre_genres',
    # 'rating_avg',
    # 'rating_votes',
    'stars',
    'directors',
]
# df.columns.to_list()
feat

for f in feat:
    df[f] = df[f].replace(" ", "")

In [210]:
def combine(r):
    return r["titre_str"]+" "+r["titre_genres"]+" "+r["stars"]+" "+r["directors"]

df["one_for_all"] = df.apply(combine, axis=1)

In [211]:
def title_index(titre):
    return df[df.titre_str == titre].index[0]

def idx_title(idx):
    return df[df.index == idx]["titre_str"].values[0]

In [212]:
tidf = TfidfVectorizer()
matrix = tidf.fit_transform(
    df["titre_str"]+" "+df["titre_genres"]+" "+df["stars"]+" "+df["directors"]
)
cosine = cosine_similarity(matrix)

movies = movie
mov_idx = title_index(movies)
similar = cosine[mov_idx]
similar1 = list(enumerate(cosine[mov_idx]))
sim_scores = sorted(similar1, key=lambda x: x[1], reverse=True)

sim_mov_idx = similar.argsort()[::-1][1:11]
same_movies = df.loc[sim_mov_idx, "titre_str"]
sim_scores[1:11]
score = [i[1] for i in sim_scores]
for movies, score in zip(same_movies, score):
    print(movies, score)


random_model = RandomForestClassifier().fit(matrix, df["titre_genres"])
for idx, movies in zip(sim_mov_idx, same_movies):
    mo = matrix[idx].toarray()
    prediction = random_model.predict(mo)[0]
    confidence = np.max(random_model.predict_proba(mo))

    print("movie :", movies)
    print("confidence :", confidence)

Spider-Man 2 1.0000000000000004
Spider-Man 3 0.8752577104803178
Spider-Man: Homecoming 0.710691577932819
The Amazing Spider-Man 2 0.29287614056278277
The Amazing Spider-Man 0.29279922245128026
Oz the Great and Powerful 0.2882275754748791
Spider-Man: Into the Spider-Verse 0.25203286331009983
Spider-Man: Across the Spider-Verse 0.24396439002483136
Darkman 0.22980155632881472
John Carter 0.2246961058339178


c:\Users\morga\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\morga\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


movie : Spider-Man 2
confidence : 0.99
movie : Spider-Man 3
confidence : 0.98
movie : Spider-Man: Homecoming
confidence : 0.97
movie : The Amazing Spider-Man 2
confidence : 0.99
movie : The Amazing Spider-Man
confidence : 0.97
movie : Oz the Great and Powerful
confidence : 0.8
movie : Spider-Man: Into the Spider-Verse
confidence : 0.88
movie : Spider-Man: Across the Spider-Verse
confidence : 0.89
movie : Darkman
confidence : 0.77
movie : John Carter
confidence : 0.95
